### This notebook walks you through the **Exploratory Data Analysis** and details about the data and the problem statement of this challege.

### If you wish to see a much broader version of the same, talking about the approach on the challenge for check my other notebook:

### [**Ultimate Guide to get started on NFL Impact Det.**](https://www.kaggle.com/aravindnaidu/ultimate-guide-to-get-started-on-nfl-impact-det)

# 1. Understanding the Competition
This competition is part of the NFL’s annual 1st and Future Competition, which has been designed to spur innovation in athlete safety and performance.

The NFL is actively addressing the need for a computer vision system to detect on-field helmet impacts as part of the “Digital Athlete” platform, and the league is calling on Kagglers to help.

In this competition, it is expected to develop a computer vision model that automatically detects helmet impacts that occur on the field. The dataset is of more than one thousand definitive head impacts from thousands of game images, labelled video from the sidelines and end zones, and player tracking data.

The data also documents the position, speed, acceleration, and orientation for every player on the field during NFL games.

This competition is evaluated using a micro F1 score at an Intersection over Union (IoU) threshold of 0.35.



# 2.Understanding Data

### The dataset consists of three types of data:

* **Image Data:**
    Image Data consist of about 10,000 images and associated helmet labels. This is to be used for building a helmet detection system.

* **Video Data:**
    Video Data consists of 120 videos (60 plays) from both a sideline and endzone point of view for each play. It has been associated with helmet and helmet impact labels, which has to be used for building a helmet impact detection system.

* **Tracking Data:**
    Tracking data consists of tracking for all players in the provided 60 plays.


### Data files:

* **train_labels.csv** - Helmet tracking and collision labels for the training set.
* **sample_submission.csv** - A valid sample submission file.
* **image_labels.csv** - contains the bounding boxes corresponding to the images.
* **[train/test]_player_tracking.csv** - Each player wears a sensor that allows us to precisely locate them on the field.



### Folders:
* **/train/** contains the mp4 video files for the training plays. 
  (Both an endzone and sideline view.)
    
* **/test/** contains the videos for the test set. 
    
* **/images/** contains the additional annotated images of player helmets.

# 3.Libraries

In [ ]:
# Libraries
import numpy as np 
import pandas as pd

import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 150

import cv2
import imageio
from IPython.display import Video, display

import warnings
warnings.filterwarnings('ignore')

# 4. Importing Data

In [ ]:
train_tracking = pd.read_csv('../input/nfl-impact-detection/train_player_tracking.csv')
test_tracking = pd.read_csv('../input/nfl-impact-detection/test_player_tracking.csv')


train_labels = pd.read_csv('../input/nfl-impact-detection/train_labels.csv')
image_labels = pd.read_csv('../input/nfl-impact-detection/image_labels.csv')
video_labels = pd.read_csv('/kaggle/input/nfl-impact-detection/train_labels.csv')

sub_sample = pd.read_csv('../input/nfl-impact-detection/sample_submission.csv')

# 5. Exploratory Data Analysis

### Number of unique elements in each feature

In [ ]:
train_labels.nunique().to_frame().rename(columns={0:"Count"})

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

sns.distplot(train_labels["gameKey"].value_counts(), ax=ax[0, 0], rug=True, color="red")
ax[0, 0].set_title("Game Counts")

sns.distplot(train_labels["playID"].value_counts(), ax=ax[0, 1], rug=True, color="blue")
ax[0, 1].set_title("Play Counts")

sns.distplot(train_labels["label"].value_counts(), ax=ax[1, 0], rug=True, color="green")
ax[1, 0].set_title("Labels Counts")

sns.distplot(train_labels["video"].value_counts(), ax=ax[1, 1], rug=True, color="yellow")
ax[1, 1].set_title("Videos Counts")

fig.show()

Let us know the number of(unique) videos in our dataset.

In [ ]:
train_labels['video'].nunique()

120 unique videos that comprise of two views of one game play each.
Therefore, 60 gameplays with two views of each.

### Lenght of videos

In [ ]:
play_frame_count = train_labels[['gameKey','playID','frame']].drop_duplicates()[['gameKey','playID']].value_counts()

fig, ax = plt.subplots(figsize=(10, 8))
sns.distplot(play_frame_count, bins=15)
ax.set_title('Distribution of frames per video file')
plt.show()

The videos range from approximately 300 frames to 600 frames per video.

### Bounding box size
This depends on various factors like,
* The distance between player and camera.
* The camera's angle and zoom relative to the field.
* One player's helmet may be blocked from view by another player.


Here, we are taking area (width x height) of the bounding box.

In [ ]:
train_labels['area'] = train_labels['width'] * train_labels['height']
fig, ax = plt.subplots(figsize=(10, 5))

sns.distplot(train_labels['area'].value_counts(),
             bins=10)
ax.set_title('Distribution bounding box sizes')
plt.show()

### Impact Type Count
Types of Impacts recorded here are:
* Helmet
* Shoulder
* Body
* Ground
* Hand
* shoulder'

In [ ]:
train_labels['impactType'].value_counts().plot(kind='bar',title='Impact Type Count',figsize=(12, 4))

plt.show()

train_labels['impactType'].value_counts()

In [ ]:
sns.catplot(x="view", hue="impactType", col="confidence",
                data=train_labels, kind="count")

In [ ]:
sns.catplot(x="view", hue="impactType", col="visibility",
                data=train_labels, kind="count")

### Impact Occurance Percentage

In [ ]:
impact_occ = train_labels[['video','impact']].fillna(0)['impact'].mean() * 100
print(f'Of all bounding boxes, {impact_occ:0.4f}% of them involve an impact event')

### Confidence
* Possible = 1
* Definitive = 2
* Definitive and Obvious = 3

In [ ]:
train_labels['confidence'].dropna().astype('int').value_counts().plot(kind='bar',
          title='Confidence Type Label Count',
          figsize=(12, 4))
plt.show()

train_labels['confidence'].value_counts()

In [ ]:
sns.catplot(x="impactType", hue="confidence", col="view",
                data=train_labels, kind="count")

### Visibility
* Not Visible from View = 0 
* Minimum = 1 
* Visible = 2
* Clearly Visible = 3

In [ ]:
train_labels['visibility'].dropna() \
    .astype('int').value_counts() \
    .plot(kind='bar',
          title='Visibility Label Count',
          figsize=(12, 4))
plt.show()

train_labels['visibility'].value_counts()

In [ ]:
sns.catplot(x="impactType", hue="visibility", col="view",
                data=train_labels, kind="count")

# Note to the Readers

### **Motivate** and **Support** me in making myself and the community better. 
### Suggest me more upgrades, share my works and also, *UPVOTE* this effort.


## Did you upvote or comment yet? Please do... :D